# __Formulación problemas de búsqueda__

La clase abstracta `SearchProblem` servirá como una plantilla general para el proceso de formulación del problema como un problema de búsqueda. Cualquier implementación deberá heredar de esta clase

In [8]:
class SearchProblem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial, goal=None, **kwds): 
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

        
    def actions(self, node):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""        
        raise NotImplementedError
        
    def result(self, node, action): 
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError
        
    def is_goal(self, node): 
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""      
        return self.goal == node
    
    def action_cost(self, s, a, s1): 
        """Return the cost of a solution path that arrives at state s1 from
        state s via action a. If the problem
        is such that the path doesn't matter, this function will only look at
        state s1.  If the path does matter, it will consider state s1
        and action a. The default method costs 1 for every step in the path."""
        return 1

    def h(self, node):
     raise NotImplementedError 
     
def g(self, node):
 return node.path_cost()

def f(self, node):
 return self.g(node) + self.h(node)

In [9]:
class Node:
    def __init__(self, state='', parent=None, action=None, cost=1):
        self.state = state
        self.parent = parent
        self.action = action

    def states_path(self):
        if self.parent == None:
            return [self]
        return self.parent.states_path() + [self]
    
    def actions_path(self):
        if self.parent == None:
            return []
        return self.parent.actions_path() + [self.action]        

    def __eq__(self, node):
        if node:
            return self.state == node.state
        return False    

    def path_cost(self):
        total_cost = 0
        if self.parent is None:
            return total_cost
        total_cost += self.cost + self.parent.path_cost()
        return total_cost 

    def __repr__(self):
        return self.state  

## Estructuras de datos - Cola de Prioridad

In [10]:
import heapq
class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)         

# UCS

In [11]:
def expand(problem, node):
    childs = []
    for action in problem.actions(node):
        childs.append(problem.result(node, action))
    return childs    

def UCS(problem, fringe=PriorityQueue()):
    closed = []
    fringe.push(problem.initial, 0)

    while(True):
        node = fringe.pop()
        if problem.is_goal(node): 
            return node
        if node not in closed: 
            closed.append(node)
            for child in expand(problem, node):
                fringe.push(child, problem.g(child))

# VORAZ

In [12]:
def expand(problem, node):
    childs = []
    for action in problem.actions(node):
        childs.append(problem.result(node, action))
    return childs    

def voraz(problem, fringe=PriorityQueue()):
    closed = []
    fringe.push(problem.initial, problem.h(problem.initial))

    while(True):
        node = fringe.pop()
        if problem.is_goal(node): 
            return node
        if node not in closed: 
            closed.append(node)
            for child in expand(problem, node):
                fringe.push(child, problem.h(child))

# A*

In [13]:
# completar 
def a_star(problem, fringe=PriorityQueue()):
    closed = []
    fringe.push(problem.initial, problem.f(problem.initial))

    while(True):
        node = fringe.pop()
        if problem.is_goal(node): 
            return node
        if node not in closed: 
            closed.append(node)
            for child in expand(problem, node):
                fringe.push(child, problem.f(child))

# Búsqueda informada sobre un grafo simple




![Picture title](image-20230228-191924.png)

In [14]:
class SimpleGraph(SearchProblem):
  'Simple Graph as a search problem'

  def actions(self, node):
    moves = {
        'S':['move-A', 'move-D'],
        'A':['move-B', 'move-G'],
        'D':['move-B', 'move-E'],
        'B':['move-E', 'move-C'],
        'C':['move-G'],
        'E':['move-G']}
    return moves[node.state]        

  def result(self, node, action):
    new_state = {
            'move-A': 'A',
            'move-B': 'B',
            'move-C': 'C',
            'move-D': 'D',
            'move-E': 'E',
            'move-G': 'G',
            'move-S': 'S'
    }      
    new_node = Node(new_state[action], node, action)
    new_cost = self.action_cost(node,action, new_node)
    new_node.cost = new_cost
    return new_node 

  def action_cost(self, node, action, succesor):
    cost = {
        ('S', 'D'): 2,
        ('S', 'A'): 3,
        ('A', 'B'): 5,
        ('A', 'G'): 10,
        ('D', 'B'): 1,
        ('D', 'E'): 4,
        ('B', 'C'): 2,
        ('B', 'E'): 1,
        ('C', 'G'): 4,
        ('E', 'G'): 3}
    return cost[(node.state, succesor.state)]

  def h(self, node):
    heuristic = {
        'S':7,
        'A':9,
        'D':5,
        'B':4,
        'C':2,
        'E':3,
        'G':0}
    return heuristic[node.state]           
  

In [17]:
simple_graph = SimpleGraph(Node('S'), Node('G'))

print('Uniform Cost Search Algorithm:')
solution = UCS(simple_graph)
print('Action sequence:', solution.actions_path())
print('State sequence:', solution.states_path())
print('Cost:', simple_graph.g(solution))


Uniform Cost Search Algorithm:


AttributeError: 'SimpleGraph' object has no attribute 'g'

## VORAZ

In [18]:
# completar
print('Algoritmo de búsqueda voraz:')
solution = voraz(simple_graph)
print('Action sequence:', solution.actions_path())
print('State sequence:', solution.states_path())
print('Cost:', simple_graph.g(solution))


Algoritmo de búsqueda voraz:
Action sequence: ['move-D', 'move-E', 'move-G']
State sequence: [S, D, E, G]


AttributeError: 'SimpleGraph' object has no attribute 'g'

## A*

In [ ]:
# completar
print('Algoritmo A*:')
solution = a_star(simple_graph)
print('Action sequence:', solution.actions_path())
print('State sequence:', solution.states_path())
print('Cost:', simple_graph.g(solution))

Algoritmo A*:


AttributeError: 'SimpleGraph' object has no attribute 'f'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c0b0ade9-403e-4acd-8eb4-c13a981e30c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>